In [102]:
# rewrite_info_cLAD.ipy

import os
import sys
import math
import scipy
import random
import warnings
import scipy.stats
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
from scipy import optimize
from scipy.stats import norm
from sklearn import datasets
from scipy import stats as st
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler
from collections import defaultdict


warnings.filterwarnings('ignore')
# np.set_printoptions(threshold='nan')
np.set_printoptions(threshold=sys.maxsize)

%matplotlib inline
# Load data and select first column

sns.set_style("white")
sns.set_context("talk")
resolution = 10000
chr_name_list = ["chr"+str(i) for i in range(1,20)]
given_resolution = 1e+4
# cLAD_dropout = 0.01
# cLAD_dropout = 0.05
cLAD_dropout = 0.10
# cLAD_dropout = 0.15
# cLAD_dropout = 0.20

cell_name_all=[17,18,19,20,25,26,27,28,29,38,39]
cell_name_index = 29
seed_sample = cell_name_index #cell index



In [103]:
LAD_dir = "/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10.bed"
LAD_data = pd.read_csv(LAD_dir,sep='\t',header=None)
LAD_data.columns=['chr','loci_head','loci_tail','3','4','5']
LAD_data['head_resolution'] = [ int(i.loci_head//given_resolution*given_resolution) for _,i in LAD_data.iterrows()]
LAD_data['tail_resolution'] = [ int(i.loci_tail//given_resolution*given_resolution) for _,i in LAD_data.iterrows()]

#dropout
len_LAD = len(LAD_data)
LAD_kept_num = int(len_LAD*(1-cLAD_dropout))
np.random.seed(seed_sample)
LAD_kept_index = random.sample(LAD_data.index, LAD_kept_num)
LAD_data = LAD_data.loc[LAD_data.index.isin(LAD_kept_index)]
LAD_data.reset_index(drop=True, inplace=True)
print("dropout rate %s, num of cLAD, before dropout %s, after dropout %s"%(cLAD_dropout, len_LAD,len(LAD_data)))
LAD_data.head()

dropout rate 0.1, num of cLAD, before dropout 3903, after dropout 3512


,chr,loci_head,loci_tail,3,4,5,head_resolution,tail_resolution
0,chr1,3010218,3087058,.,1,.,3010000,3080000
1,chr1,3126730,3196670,.,1,.,3120000,3190000
2,chr1,3220610,3282860,.,1,.,3220000,3280000
3,chr1,3339298,3357024,.,1,.,3330000,3350000
4,chr1,3371740,3389850,.,1,.,3370000,3380000


In [104]:
windows_sep_dir = "/home/ygli/gam_paper_data/gam_seq_mapped4/bowtie_result/chrom_windows/"
# chr_name = "chr1"
# windows_sep_file = "mm10_"+chr_name+"_windows_10000.bed"
# windows_sep = pd.read_csv(windows_sep_dir+windows_sep_file, sep='\t', header=None)
# windows_sep.columns = ['chr','loci_head', 'loci_tail']
# windows_sep.head()

In [105]:
# return the shortest dis from each element in listB to listA
# return a dictionary with the same length as listB, with key as indexB_sort, value as the shorteds dis
def get_intersection_shortest_distance(listA, listB, indexB):
    listA_sort = sorted(listA)
    listB_sort = sorted(listB)
    indexB_sort = [x for (y,x) in sorted(zip([i[0] for i in listB], indexB))]
    inter_index = []
    inter_dis = defaultdict(list)
    i,j = 0,0
    while i < len(listA_sort) and j < len(listB_sort):
#     while j < len(listB_sort):
        l = max(listA_sort[i][0], listB_sort[j][0])
        r = min(listA_sort[i][1], listB_sort[j][1])
        if l <= r:
            inter_index.append(indexB_sort[j])
            inter_dis[indexB_sort[j]].append(0) #intersection
        else:
            inter_dis[indexB_sort[j]].append(l-r) #this two eles not intersection
        if listA_sort[i][1] < listB_sort[j][1]:
            i += 1
        else:
            j += 1
    while j < len(listB_sort):
        l = max(listA_sort[-1][0], listB_sort[j][0])
        r = min(listA_sort[-1][1], listB_sort[j][1])
        inter_dis[indexB_sort[j]].append(l-r)
        j+=1
    
    inter_dis = {i:np.min(inter_dis[i]) for i in inter_dis}
    return inter_dis

# listA = [[4,7], [1,3], [5,9]]
# # listB = [[20,50], [2,4], [5,8], [0,3]]
# listB = [[1,3], [3,6], [6,9], [12,15]]
# indexB = [1,2,4,0]
# get_intersection_index(listA, listB, indexB)



In [106]:
save_dir = "/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/"
# chr_name = "chr1"

for chr_name in chr_name_list:
# for chr_name in ['chr1']:
    if cLAD_dropout==0:
        save_file_name = chr_name+"_shortest_distance.csv"
    else:
        save_file_name = chr_name+"_shortest_distance_dpot"+str(cLAD_dropout)+"_seed"+str(seed_sample)+".csv"
#     save_file_name = chr_name+"_shortest_distance.csv"
    windows_sep_file = "mm10_"+chr_name+"_windows_10000.bed"
    windows_sep = pd.read_csv(windows_sep_dir+windows_sep_file, sep='\t', header=None)
    windows_sep.columns = ['chr','loci_head', 'loci_tail']

    listA = [[i.loci_head, i.loci_tail] for _,i in LAD_data[LAD_data.chr==chr_name].iterrows()]
    listB = [[i.loci_head, i.loci_tail] for _,i in windows_sep.iterrows()]
    indexB = [i for i,_ in windows_sep.iterrows()]
    intersection_shortest_distance_chr = get_intersection_shortest_distance(listA, listB, indexB)
    intersection_shortest_distance_chr_list = [intersection_shortest_distance_chr[i] for i in intersection_shortest_distance_chr]
    head_lociB = [listB[i][0] for i in intersection_shortest_distance_chr]
    tail_lociB = [listB[i][1] for i in intersection_shortest_distance_chr]
    intersection_shortest_distance_chr_df = pd.DataFrame({
        "head_loci": head_lociB,
        "tail_loci": tail_lociB,
        "shortest_distance": intersection_shortest_distance_chr_list,
    })
#     print(intersection_shortest_distance_chr_df.head())
    intersection_shortest_distance_chr_df.to_csv(save_dir+save_file_name, columns=['head_loci', 'tail_loci', 'shortest_distance'], index=0)
    print(save_dir+save_file_name)


/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/chr1_shortest_distance_dpot0.1_seed29.csv
/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/chr2_shortest_distance_dpot0.1_seed29.csv
/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/chr3_shortest_distance_dpot0.1_seed29.csv
/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/chr4_shortest_distance_dpot0.1_seed29.csv
/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/chr5_shortest_distance_dpot0.1_seed29.csv
/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/chr6_shortest_distance_dpot0.1_seed29.csv
/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/chr7_shortest_distance_dpot0.1_seed29.csv
/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/chr8_shortest_distance_dpot0.1_seed29.csv
/home/ygli/gam_paper_data/lad_data/GSE17051_cLAD_regions_mm10_10000/chr9_shortest_distance_dpot0.1_seed29.csv
/home/ygli

In [101]:
intersection_shortest_distance_chr_df.shape

(6144, 3)

In [66]:
windows_sep.shape

(6144, 3)